# Workflow
![shape-based approach](img_embed/w8_circle_detection.webp)

# Circles detection

## Step 0: Setup

In [2]:
from utils import display_image, display_images, auto_canny
import cv2 as cv
import numpy as np

## Step 1: Load image

In [3]:
img = cv.imread("images/w8/circles/clock.jpg")
display_image("image", img)

## Step 2: Define parameter settings

In [9]:
max_width = 400
ksize = 7
canny_method = "triangle"
min_circularity = 0.85
min_area = 400

## Step 3: Major workflow diagram

In [4]:
# define a utility function to filter contour by area
def filterFunc(x):
    """Filter function to filter x
    Arguments:
    ---
    x: list of contours (outputs of cv.findContours())
    
    Returns:
    ---
    Boolean"""
    if cv.contourArea(x) > min_area:
        return True
    else:
        return False

In [11]:
# main function
def circle_detector(img, min_circularity=min_circularity, canny_method=canny_method, ksize=ksize):
    """Function that can detect circular objects
    Arguments:
    ---
    img: source image (uint8)
    min_circularity: minimum threshold of circularity
    canny_method: auto_canny methods: "triangle", "otsu" or "median"
    ksize: Gaussian blur kernel size
    Returns:
    ---
    contours of circular objects"""
    # copy
    # img_copy = img.copy()
    # resize
    # f = max_width / img.shape[1]
    # img = cv.resize(img, None, fx=f, fy=f, interpolation=cv.INTER_CUBIC)

    # change to grayscale
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # blur
    gray = cv.GaussianBlur(gray, (ksize, ksize), 0)
    # canny edge detection
    edge = auto_canny(gray, method=canny_method)

    # Contour
    contours, _ = cv.findContours(edge, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    # filter by area and circularity
    # contour = contours[0]
    contours_filtered = filter(filterFunc, contours)

    circular_obj = []
    for cnt in contours_filtered:
        area = cv.contourArea(cnt)
        peri = cv.arcLength(cnt, True)
        try:
            circularity = (4 * np.pi * area) / peri
            if circularity > min_circularity:
                circular_obj.append(cnt)
        except:
            continue
            
    # cv.drawContours(img_resized, cnts, -1, (0, 255, 0), 2)
    return circular_obj

## Step 4: Run the pipeline on all the images

In [12]:
import os

img_dir = "./images/w8/circles/"
file_names = os.listdir(img_dir)

for filename in file_names:
    filepath = os.path.join(img_dir, filename)
    img = cv.imread(cv.samples.findFile(filepath))
    f = max_width / img.shape[1]
    img = cv.resize(img, None, fx=f, fy=f, interpolation=cv.INTER_CUBIC)
    img_copy = img.copy()
    cnts = circle_detector(img_copy)
    cv.drawContours(img, cnts, -1, (0, 255, 0), 2)
    cv.putText(img, f"# circles detected: {len(cnts)}", (20, 20), cv.FONT_HERSHEY_SIMPLEX,
              0.5, (0, 255, 0))
    display_images([img_copy, img], ("resized", "detect circles"))
    cv.imwrite(img_dir + f"{filename}_res.jpg", img)

img_dir = ""

# Circle detector (alternate method: Hough Transform)

In [26]:
# Hough transform parameters
dp = 4   # accumulator resolution
param1 = 400  # upper Canny threshold
param2 = 0.97   # perfectness of cirles
minRadius = 50

ksize = 5
max_width = 500

# minimum distance between circles and maximum radius of circles depends on the image width
def circle_hough_detector(img, ksize=ksize):
    """Hough circle transform"""
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray, (ksize, ksize), 0)

    width = gray.shape[1]
    circles = cv.HoughCircles(blur, cv.HOUGH_GRADIENT_ALT, dp, width / 6, 
                             param1=param1, param2=param2, minRadius=minRadius, maxRadius=int(0.4*width))
    if circles is not None:
        return circles[0]

In [27]:
import os

img_dir = "./images/w8/circles/"
file_names = os.listdir(img_dir)

for filename in file_names:
    filepath = os.path.join(img_dir, filename)
    img = cv.imread(cv.samples.findFile(filepath))
    f = max_width / img.shape[1]
    img = cv.resize(img, None, fx=f, fy=f, interpolation=cv.INTER_CUBIC)
    img_copy = img.copy()
    circles = circle_hough_detector(img_copy)
    if circles is not None:
        circles = np.uint16(np.round(circles))
        
        for i in circles:
            center = (i[0], i[1])
            cv.circle(img_copy, center, 1, (0, 0, 255), 1)
            radius = i[2]
            cv.circle(img_copy, center, radius, (100, 50, 200), 2)
    cv.putText(img_copy, f"# circles: {len(circles)}", (30, 30), 
              cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    display_image(f"{filename} result", img_copy)

img_dir = ""

# Rectangle detection

## Step 0: Setup

In [1]:
from utils import display_image, display_images, auto_canny, resize_aspect_ratio
import cv2 as cv
import numpy as np

In [14]:
# def enhance_contrast(img):
#     """"""
#     img_lab = cv.cvtColor(img, cv.COLOR_BGR2LAB)
#     l, a, b = cv.split(img_lab)
#     # clahe = cv.createCLAHE(clipLimit=10, tileGridSize=(20, 20))
#     # l_enhanced = clahe.apply(l)
#     l_enhanced = cv.equalizeHist(l)
#     img_enhanced = cv.merge((l_enhanced, a, b))
#     return cv.cvtColor(img_enhanced, cv.COLOR_LAB2BGR)

## Step 1: Load image

In [2]:
img = cv.imread("images/w8/rectangles/ipad.jpg")
display_image("image", img)

## Step 3: Setup pipeline
1. Load image and visualize
2. Resize
3. Contrast enhancement
4. Grayscale
5. Automatic Canny edge detector
6. Contour detection and post processing
7. Draw contour and insert text to show how many rectangular object is detected.

In [3]:
min_area = 600
def filterFunc(x):
    """Filter function to filter x
    Arguments:
    ---
    x: list of contours (outputs of cv.findContours())
    
    Returns:
    ---
    Boolean"""
    if cv.contourArea(x) > min_area:
        return True
    else:
        return False

In [7]:
max_width = 500
canny_method = "median"
frac_peri = 0.02

In [8]:
def rect_detector(img, max_width=max_width, canny_method=canny_method, frac_peri=frac_peri, debug=False):
    """"""
    # resize
    resized = resize_aspect_ratio(img, width=max_width)
    resized_copy = resized.copy()
    # enhance contrast
    # resized = cv.GaussianBlur(resized, (9, 9), 0)
    resized = cv.pyrMeanShiftFiltering(resized, 21, 41)
    resized = cv.GaussianBlur(resized, (3, 3), 0)
    if debug:
        display_images([img, resized], ("original", "ce"))
    # grayscale
    gray = cv.cvtColor(resized, cv.COLOR_BGR2GRAY)
    # Canny
    edge = auto_canny(gray, method=canny_method)
    if debug:
        display_image("edge", edge)
    # Contour
    contours, _ = cv.findContours(edge, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    contours_filtered = filter(filterFunc, contours)
    
    rect_cnts = []
    for cnt in contours_filtered:
        if cv.contourArea(cnt) <= min_area:
            continue
        else:
            peri = cv.arcLength(cnt, True)
            approx = cv.approxPolyDP(cnt, frac_peri * peri, True)
        
            if len(approx) == 4:
                rect_cnts.append(approx)
    
    cv.drawContours(resized_copy, rect_cnts, -1, (0, 255, 0))
    if debug:
        display_image("contour", resized)
    return resized_copy, rect_cnts

In [9]:
import os

img_dir = "./images/w8/rectangles/"
filenames = os.listdir(img_dir)

for file in filenames:
    filepath = os.path.join(img_dir, file)
    img = cv.imread(cv.samples.findFile(filepath))
    dst, rects = rect_detector(img)
    item = file.split(".")[0]
    cv.putText(dst, f"{item} detected: {len(rects)}", (25, 25), 
              cv.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    display_images([img, dst], ("original", "detect"))
    file_save_name = f"{item}_res.jpg"
    cv.imwrite(img_dir + file_save_name, dst)

img_dir = ""